# Space Navigator with Collinear Grid Search

In [1]:
import numpy as np

from space_navigator.models.CE import CrossEntropy
from space_navigator.models.collinear_GS import CollinearGridSearch
from space_navigator.utils import read_environment

In [2]:
env_path = "../../../data/environments/collision.env"
env = read_environment(env_path)

# Training

### Init

In [3]:
# __init__ docstring
print(CollinearGridSearch.__init__.__doc__)


        Agrs:
            env (Environment): environment with given parameteres.
            step (float): time step in simulation.
            reverse (bool): 
                if True: there are selected exactly 2 maneuvers
                    while the second of them is reversed to the first one;
                if False: one maneuver.
            first_maneuver_direction (str): first maneuver is collinear
                to the velocity vector and could be:
                    "forward" (co-directed)
                    "backward" (oppositely directed)
                    "auto" (just collinear).
        


In [4]:
init_parameters = {
    "env": env,
    "step": 1e-6,
    "reverse": True, 
}

In [5]:
collinear_GS_model = CollinearGridSearch(**init_parameters)

### Iteration

(currently it is just one iteration)

In [6]:
# iteration docstring
print(collinear_GS_model.iteration.__doc__)

Training iteration.

        Args:
            print_out (bool): print iteration information.
            n_sessions (int): number of sessions to generate.

        Returns:
            stop (bool): whether to stop training after iteration.

        


In [7]:
iteration_parameters = {
    "n_sessions": 1000,
}

In [8]:
# one iteration
# collinear_GS_model.iteration(**iteration_parameters)

### Train

In [9]:
# train docstring
print(collinear_GS_model.train.__doc__)

Training agent policy (self.action_table).

        Args:
            n_iterations (int): number of iterations.
            print_out (bool): print information during the training.
            *args and **kwargs: iteration arguments, depend on method (inheritor class).

        TODO:
            add early stopping
            add log
            decorate by print_out and log?
        


In [10]:
n_iterations = 1
print_out = True

In [11]:
collinear_GS_model.train(n_iterations, print_out, **iteration_parameters)

  1%|          | 11/1000 [00:00<00:09, 107.68it/s]


Start training.

Initial action table:
[]
Initial Reward: -13489.530475182863

iteration: 1/1


100%|██████████| 1000/1000 [00:09<00:00, 105.58it/s]


Training completed in 9.4744 sec.
Total Reward: -0.5260470455285451
Action Table:
[[ 0.          0.          0.          0.        ]
 [ 1.60913022  1.51055235  0.02636679  0.07934858]
 [-1.60913022 -1.51055235 -0.02636679         nan]]


In [12]:
# obtained table of actions
collinear_GS_model.action_table

array([[ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 1.60913022,  1.51055235,  0.02636679,  0.07934858],
       [-1.60913022, -1.51055235, -0.02636679,         nan]])

In [13]:
# save_path =
# collinear_GS_model.save_action_table(save_path)

# Tuning

In [14]:
CE_init_parameters = {
    "env": env,
    "step": 1e-6,
    "reverse": True, 
    "first_maneuver_time": 'early',
    "n_maneuvers": 2,
    "lr": 0.9,
    "percentile": 95,
    "sigma_dV": None,
    "sigma_t": None,
}
CE_iteration_parameters = {
    "n_sessions": 100,
    "sigma_decay": 0.9,
    "lr_decay": 0.9,
    "percentile_growth": 1.01,
    "show_progress": False,
    "dV_angle": 'complanar',
    "step_if_low_reward": False,
    "early_stopping": True,
}

In [15]:
tune_CE_model = CrossEntropy(**init_parameters)

In [16]:
n_iterations = 50
print_out = True

# set initial actions table for tuning
tune_CE_model.set_action_table(collinear_GS_model.action_table)
# tuning
tune_CE_model.train(n_iterations, print_out, **CE_iteration_parameters)

print(f"Actions table after tuning:\n{tune_CE_model.action_table}")
print(f"\nReward: {tune_CE_model.policy_reward}")

  8%|▊         | 8/100 [00:00<00:01, 76.83it/s]


Start training.

Initial action table:
[[ 0.          0.          0.          0.        ]
 [ 1.60913022  1.51055235  0.02636679  0.07934858]
 [-1.60913022 -1.51055235 -0.02636679         nan]]
Initial Reward: -0.5260470455285451

iteration: 1/50


 10%|█         | 10/100 [00:00<00:00, 95.32it/s]

Policy Reward: -0.5260470455285451
Mean Reward:   -233.1354991146465
Max Reward:    -15.547391400031907
Threshold:     -35.594369310229496

iteration: 2/50


 10%|█         | 10/100 [00:00<00:00, 95.06it/s]

...

Policy Reward: -0.5258205239009067
Mean Reward:   -42.947304519446334
Max Reward:    -0.5292041812633463
Threshold:     -0.7133077016589888

iteration: 19/50


 11%|█         | 11/100 [00:00<00:00, 102.51it/s]

Policy Reward: -0.5258205239009067
Mean Reward:   -44.61188542629712
Max Reward:    -0.6836780165430262
Threshold:     -0.7947430344716759

iteration: 20/50


100%|██████████| 100/100 [00:00<00:00, 101.19it/s]

Policy Reward: -0.5258205239009067
Mean Reward:   -45.424706776887234
Max Reward:    -0.5288992337292051
Threshold:     -0.67696453540506

Early stopping.

Training completed in 20.618 sec.
Total Reward: -0.5258205239009067
Action Table:
[[ 0.          0.          0.          0.        ]
 [ 1.60131478  1.51420806  0.0264306   0.07934858]
 [-1.60131478 -1.51420806 -0.0264306          nan]]
Actions table after tuning:
[[ 0.          0.          0.          0.        ]
 [ 1.60131478  1.51420806  0.0264306   0.07934858]
 [-1.60131478 -1.51420806 -0.0264306          nan]]

Reward: -0.5258205239009067
